## weight-tuning_LG_UA

Get an acceptable SLFN first.

*   Learning Goal : loss要小於0.4
*   Undesired Attractor : learning rate 不能小於 0.001

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load the Boston Housing dataset
data = fetch_california_housing()

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(data.data)
y = data.target.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# 將數據轉換為 PyTorch 張量
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
# 設定參數
D_in = X.shape[1]
H = 11
D_out = 1

# learning rate
learning_rate = 0.1

# when step size < 0.001, stop training => Unacceptable
threshold = 0.001

In [ ]:
from collections import OrderedDict
# Define 2-layer neural network
model_LG_UA = torch.nn.Sequential(OrderedDict([
    ('fc1', torch.nn.Linear(D_in, H)),
    ('tanh', torch.nn.Tanh()),
    ('output', torch.nn.Linear(H, D_out))
])
)

# 定義優化器和損失函數
criterion = nn.MSELoss()
optimizer = optim.Adam(model_LG_UA.parameters(), lr=learning_rate)

In [ ]:
train_losses = []
# 訓練模型
b_loss = float('Inf')
epoch = 0
while b_loss > 0.4: # traing loss要小於我設定的stopping criteria才會停
  # 第一次forward
  optimizer.zero_grad()
  output_1 = model_LG_UA(X_train) # w
  torch.save(model_LG_UA.state_dict(), 'save.pt') # 存下w的參數
  w_loss = criterion(output_1, y_train) # Loss(w)
  w_loss.backward() # w的偏微分

  while optimizer.param_groups[0]['lr'] > threshold:
    # optimizer.step() => According to learning rate and other parameters，來更新我們的參數。
    optimizer.step() # w'：w往偏微分的方向走一個step size

    # 第二次forward
    output_2 = model_LG_UA(X_train) # w'
    ww_loss = criterion(output_2, y_train) # Loss(w')

    # 這裡要判斷 : Loss(w') 是否小於 Loss(w)
    if ww_loss < w_loss:
      # 目前參數：w'
      # step size變大
      optimizer.param_groups[0]['lr'] *= 1.2
      b_loss = ww_loss
      # Collect the training loss values in list
      train_losses.append(ww_loss.item())
      epoch += 1
      '''if epoch % 1 == 0:
        print('Epoch {}, Loss: {:.4f}'.format(epoch, b_loss))'''
      break
    else:
      # 此時 step size 一定大於 threshold
      # 參數要是：w
      model_LG_UA.load_state_dict(torch.load("save.pt"))
      # step size *0.7
      optimizer.param_groups[0]['lr'] *= 0.7
      b_loss = ww_loss
      # Collect the training loss values in list
      train_losses.append(ww_loss.item())
      epoch += 1
      # print('Epoch {}, Loss: {:.4f} > {:.4f}'.format(epoch, b_loss, w_loss.item()))

  '''if epoch % 100 == 0:
      print('Epoch {}, Loss: {:.4f}'.format(epoch+1, b_loss))'''

  if optimizer.param_groups[0]['lr'] <= threshold:
    print('\nUnacceptable because learning rate too small, Epoch {}, Loss: {:.4f}, Step size: {}'.format(epoch, b_loss, optimizer.param_groups[0]['lr']))
    break

if b_loss < 0.4:
  print('\nAcceptable, Epoch {}, Loss: {}'.format(epoch, b_loss))

epsilon = b_loss.item()
print('epsilon = ', epsilon)


Acceptable, Epoch 103, Loss: 0.3996618986129761
epsilon =  0.3996618986129761


In [ ]:
torch.save(model_LG_UA, 'Acc_SLFN.pt') # 存下 weight-tuning_LG_UA 的所有參數，當作 network-tuning 的 Acceptable SLFN

In [ ]:
params_list = list(model_LG_UA.named_parameters())
# Print the list
for name, param in params_list:
    print(f'Parameter {name}: {param}')

Parameter fc1.weight: Parameter containing:
tensor([[-0.2829, -0.0460, -0.5183, -1.7815,  0.0937,  4.8707,  0.3505,  0.5282],
        [-0.5507, -0.2610,  0.8829, -0.1531,  0.8875,  2.1367, -0.3348,  0.8159],
        [-0.2699, -1.3000, -0.3128,  2.2418,  0.7542,  0.6323, -0.0692,  1.1013],
        [-0.7228, -0.4929,  0.4781,  0.4822,  0.2418, -0.4008,  1.8225,  0.2303],
        [ 0.5856,  0.3874, -0.1952,  0.9962,  0.2507,  0.6442, -1.5841, -1.2807],
        [-0.3635,  0.4361,  0.3293, -0.4820, -0.1381,  2.2665,  0.6198,  0.0907],
        [-0.5631,  0.6171,  0.6116, -0.0374,  0.1054,  3.2073, -0.2021,  0.7409],
        [-1.1067, -0.7602, -0.3881, -1.8475, -0.2217, -0.3042, -2.0198,  0.3514],
        [ 0.2729,  0.0596, -0.2209, -1.4443, -0.0147,  0.5396,  0.1975,  0.2928],
        [-0.4659,  1.0877, -1.5324,  0.1719, -0.0927,  0.8680, -0.1611,  0.5407],
        [ 0.8690,  0.7839, -0.3650, -0.0778,  0.0733, -0.3152, -1.9519,  0.0330]],
       requires_grad=True)
Parameter fc1.bias: Parame

## network-tuning_1

if k <= p:

step 1. 從第 k=1 個hidden node開始刪除

step 2. 檢查刪除後的 model 是否滿足 learning goal

step 3. 

*   if step 2. true: do regularizing, k 不加1, p-1, and goto step 1.
*   if step 2. false: restore weight, k+1, p不加1, and do regularizing. Then goto step 1.

In [ ]:
import numpy as np
model_1 = torch.load('Acc_SLFN.pt')

In [ ]:
p = H
k = 1
count = 0
while k <= p:
  torch.save(model_1, 'temp.pt')
  # Get the weight matrix of the first linear layer of the model
  weight_matrix = model_1.fc1.weight.detach().numpy()
  # Delete the k-th row from the weight matrix using NumPy's delete() function
  modified_weight_matrix = np.delete(weight_matrix, (k-1), axis=0)
  model_1.fc1.weight = nn.Parameter(torch.from_numpy(modified_weight_matrix).type(torch.Tensor))

  bias_matrix = model_1.fc1.bias.detach().numpy()
  modified_bias_matrix = np.delete(bias_matrix, k-1)
  model_1.fc1.bias = nn.Parameter(torch.from_numpy(modified_bias_matrix).type(torch.Tensor))

  weight_matrix_2 = model_1.output.weight.detach().numpy()
  modified_weight_matrix_2 = np.delete(weight_matrix_2, k-1, 1)
  model_1.output.weight = nn.Parameter(torch.from_numpy(modified_weight_matrix_2).type(torch.Tensor))

  output = model_1(X_train)
  w_loss = criterion(output, y_train)
  w_loss.backward() # w的偏微分
  if w_loss > 0.4:
    print(f'\nRestore 第{count+1}個 hidden node, because it is an unacceptable SLFN')
    model_1 = torch.load('temp.pt')

    # regularizing
    optimizer.param_groups[0]['lr'] *= 0.7
    optimizer.step() # w'：w往偏微分的方向走一個step size
    print('Regularizing done.')
    k += 1
    count += 1
  else:
    print(f'\nDelete 第{count+1}個 hidden node, the loss = {w_loss:.4f}')
    print('Acceptable SLFN')
    # regularizing
    optimizer.param_groups[0]['lr'] *= 0.7
    optimizer.step() # w'：w往偏微分的方向走一個step size
    print('Regularizing done.')
    p -= 1
    count += 1


Restore 第1個 hidden node, because it is an unacceptable SLFN
Regularizing done.

Restore 第2個 hidden node, because it is an unacceptable SLFN
Regularizing done.

Restore 第3個 hidden node, because it is an unacceptable SLFN
Regularizing done.

Restore 第4個 hidden node, because it is an unacceptable SLFN
Regularizing done.

Restore 第5個 hidden node, because it is an unacceptable SLFN
Regularizing done.

Restore 第6個 hidden node, because it is an unacceptable SLFN
Regularizing done.

Restore 第7個 hidden node, because it is an unacceptable SLFN
Regularizing done.

Restore 第8個 hidden node, because it is an unacceptable SLFN
Regularizing done.

Restore 第9個 hidden node, because it is an unacceptable SLFN
Regularizing done.

Restore 第10個 hidden node, because it is an unacceptable SLFN
Regularizing done.

Restore 第11個 hidden node, because it is an unacceptable SLFN
Regularizing done.


In [ ]:
params_list = list(model_1.named_parameters())
# Print the list
for name, param in params_list:
    print(f'Parameter {name}: {param}')

Parameter fc1.weight: Parameter containing:
tensor([[-0.2829, -0.0460, -0.5183, -1.7815,  0.0937,  4.8707,  0.3505,  0.5282],
        [-0.5507, -0.2610,  0.8829, -0.1531,  0.8875,  2.1367, -0.3348,  0.8159],
        [-0.2699, -1.3000, -0.3128,  2.2418,  0.7542,  0.6323, -0.0692,  1.1013],
        [-0.7228, -0.4929,  0.4781,  0.4822,  0.2418, -0.4008,  1.8225,  0.2303],
        [ 0.5856,  0.3874, -0.1952,  0.9962,  0.2507,  0.6442, -1.5841, -1.2807],
        [-0.3635,  0.4361,  0.3293, -0.4820, -0.1381,  2.2665,  0.6198,  0.0907],
        [-0.5631,  0.6171,  0.6116, -0.0374,  0.1054,  3.2073, -0.2021,  0.7409],
        [-1.1067, -0.7602, -0.3881, -1.8475, -0.2217, -0.3042, -2.0198,  0.3514],
        [ 0.2729,  0.0596, -0.2209, -1.4443, -0.0147,  0.5396,  0.1975,  0.2928],
        [-0.4659,  1.0877, -1.5324,  0.1719, -0.0927,  0.8680, -0.1611,  0.5407],
        [ 0.8690,  0.7839, -0.3650, -0.0778,  0.0733, -0.3152, -1.9519,  0.0330]],
       requires_grad=True)
Parameter fc1.bias: Parame

## network-tuning_2

if k <= p:

step 1. 從第 k=1 個hidden node開始刪除

step 2. weight-tuning 刪掉hidden node 的 model

step 3. 判斷是否到達learning goal

*   if yes: do regularizing, k 不加1, p-1, and goto step 1.
*   if no: restore weight, k+1, p不加1, and do regularizing. Then goto step 1.

In [ ]:
model_2 = torch.load('Acc_SLFN.pt')

In [ ]:
p = H
k = 1
count = 0
while k <= p:
  torch.save(model_2, 'temp.pt')
  # Get the weight matrix of the first linear layer of the model
  weight_matrix = model_2.fc1.weight.detach().numpy()
  # Delete the k-th row from the weight matrix using NumPy's delete() function
  modified_weight_matrix = np.delete(weight_matrix, (k-1), axis=0)
  model_2.fc1.weight = nn.Parameter(torch.from_numpy(modified_weight_matrix).type(torch.Tensor))

  bias_matrix = model_2.fc1.bias.detach().numpy()
  modified_bias_matrix = np.delete(bias_matrix, k-1)
  model_2.fc1.bias = nn.Parameter(torch.from_numpy(modified_bias_matrix).type(torch.Tensor))

  weight_matrix_2 = model_2.output.weight.detach().numpy()
  modified_weight_matrix_2 = np.delete(weight_matrix_2, k-1, 1)
  model_2.output.weight = nn.Parameter(torch.from_numpy(modified_weight_matrix_2).type(torch.Tensor))

  # weight-tuning
  for epoch in range(100):
    optimizer.zero_grad()
    output = model_2(X_train)
    w_loss = criterion(output, y_train)
    if w_loss > 0.4:
      w_loss.backward() # w的偏微分
      optimizer.step()
    else:
      print(f'\nDelete 第{count+1}個 hidden node and weight-tuning to an acceptable SLFN')
      # regularizing
      optimizer.param_groups[0]['lr'] *= 0.7
      optimizer.step() # w'：w往偏微分的方向走一個step size
      print('Regularizing done.')
      p -= 1
      count += 1
      break
    
  if epoch+1 == 100 and w_loss > 0.4:
    print(f'\nRestore 第{count+1}個 hidden node, because it can not weight-tuning to an acceptable SLFN')
    model_2 = torch.load('temp.pt')

    # regularizing
    optimizer.param_groups[0]['lr'] *= 0.7
    optimizer.step() # w'：w往偏微分的方向走一個step size
    print('Regularizing done.')
    k += 1
    count += 1


Restore 第1個 hidden node, because it can not weight-tuning to an acceptable SLFN
Regularizing done.

Restore 第2個 hidden node, because it can not weight-tuning to an acceptable SLFN
Regularizing done.

Restore 第3個 hidden node, because it can not weight-tuning to an acceptable SLFN
Regularizing done.

Restore 第4個 hidden node, because it can not weight-tuning to an acceptable SLFN
Regularizing done.

Restore 第5個 hidden node, because it can not weight-tuning to an acceptable SLFN
Regularizing done.

Restore 第6個 hidden node, because it can not weight-tuning to an acceptable SLFN
Regularizing done.

Restore 第7個 hidden node, because it can not weight-tuning to an acceptable SLFN
Regularizing done.

Restore 第8個 hidden node, because it can not weight-tuning to an acceptable SLFN
Regularizing done.

Restore 第9個 hidden node, because it can not weight-tuning to an acceptable SLFN
Regularizing done.

Restore 第10個 hidden node, because it can not weight-tuning to an acceptable SLFN
Regularizing done.

## network-tuning_3

step 1. if fail小於3: Random 刪掉 1 個 hidden node

step 2. weight-tuning the model

step 3. 

*   if step 2. 到達learning goal: use regularizing retrain model and goto step 1.
*   if step 2. 無法到達learning goal: fail++ and restore weight and goto step 1.

In [ ]:
model_3 = torch.load('Acc_SLFN.pt')

In [ ]:
import random

my_set = set(range(11))
fail = 0
while fail < 3 and bool(my_set):
  k = random.sample(my_set, 1)[0]
  my_set.remove(k)
  print('\n')
  print(k)
 
  # 刪除hidden node之前，先記下目前的model
  torch.save(model_3, 'temp.pt')
  # Get the weight matrix of the first linear layer of the model
  weight_matrix = model_3.fc1.weight.detach().numpy()
  # Delete the k-th row from the weight matrix using NumPy's delete() function
  modified_weight_matrix = np.delete(weight_matrix, (k), axis=0)
  model_3.fc1.weight = nn.Parameter(torch.from_numpy(modified_weight_matrix).type(torch.Tensor))

  bias_matrix = model_3.fc1.bias.detach().numpy()
  modified_bias_matrix = np.delete(bias_matrix, k)
  model_3.fc1.bias = nn.Parameter(torch.from_numpy(modified_bias_matrix).type(torch.Tensor))

  weight_matrix_2 = model_3.output.weight.detach().numpy()
  modified_weight_matrix_2 = np.delete(weight_matrix_2, k, 1)
  model_3.output.weight = nn.Parameter(torch.from_numpy(modified_weight_matrix_2).type(torch.Tensor))

  # weight-tuning
  for epoch in range(100):
    optimizer.zero_grad()
    output = model_3(X_train)
    w_loss = criterion(output, y_train)
    if w_loss > 0.4:
      w_loss.backward() # w的偏微分
      optimizer.step()
    else:
      print(f'Delete this hidden node and weight-tuning to an acceptable SLFN')
      # regularizing
      optimizer.param_groups[0]['lr'] *= 0.7
      optimizer.step() # w'：w往偏微分的方向走一個step size
      print('Regularizing done.')
      my_set.remove(max(my_set)) # 刪除最大的數，因為長度-1
      break

    if epoch+1 == 100 and w_loss > 0.4:
      print(f'Restore this hidden node, because it can not weight-tuning to an acceptable SLFN')
      model_3 = torch.load('temp.pt')

      # regularizing
      optimizer.param_groups[0]['lr'] *= 0.7
      optimizer.step() # w'：w往偏微分的方向走一個step size
      print('Regularizing done.')
      fail += 1
      



5


<ipython-input-14-2eda94bd14fd>:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  k = random.sample(my_set, 1)[0]


Restore this hidden node, because it can not weight-tuning to an acceptable SLFN
Regularizing done.


2
Restore this hidden node, because it can not weight-tuning to an acceptable SLFN
Regularizing done.


7
Restore this hidden node, because it can not weight-tuning to an acceptable SLFN
Regularizing done.


## network-tuning_4

step 1. store the network with *p* hidden nodes and its weights *w*.

step 2. use the magnitude of the weights connecting the hidden nodes to the output as a measure of importance.

step 3. remove the least important nodes.

step 4. let *p-1* replaced to *p*, and $w_{k}' = w - \{w_{k}^0, w_{k0}^H, w_{k}^H \}$

step 5. create the new network

In [ ]:
model_4 = torch.load('Acc_SLFN.pt')

In [ ]:

# 刪除hidden node之前，先記下目前的model
torch.save(model_4, 'temp.pt')
importance = []
for i in range(H):
  a = np.linalg.norm(model_4.fc1.weight.detach().numpy()[i])
  importance.append(a)
less_importance_id = importance.index(min(importance))

# Get the weight matrix of the first linear layer of the model
weight_matrix = model_4.fc1.weight.detach().numpy()
# Delete the k-th row from the weight matrix using NumPy's delete() function
modified_weight_matrix = np.delete(weight_matrix, (less_importance_id), axis=0)
model_4.fc1.weight = nn.Parameter(torch.from_numpy(modified_weight_matrix).type(torch.Tensor))

bias_matrix = model_4.fc1.bias.detach().numpy()
modified_bias_matrix = np.delete(bias_matrix, less_importance_id)
model_4.fc1.bias = nn.Parameter(torch.from_numpy(modified_bias_matrix).type(torch.Tensor))

weight_matrix_2 = model_4.output.weight.detach().numpy()
modified_weight_matrix_2 = np.delete(weight_matrix_2, less_importance_id, 1)
model_4.output.weight = nn.Parameter(torch.from_numpy(modified_weight_matrix_2).type(torch.Tensor))

# weight-tuning
for epoch in range(100):
  optimizer.zero_grad()
  output = model_4(X_train)
  w_loss = criterion(output, y_train)
  if w_loss > 0.4:
    w_loss.backward() # w的偏微分
    optimizer.step()
  else:
    print(f'Delete this hidden node and weight-tuning to an acceptable SLFN')
    # regularizing
    optimizer.param_groups[0]['lr'] *= 0.7
    optimizer.step() # w'：w往偏微分的方向走一個step size
    print('Regularizing done.')
    break

  if epoch+1 == 100 and w_loss > 0.4:
    print(f'Restore this hidden node, because it can not weight-tuning to an acceptable SLFN')
    model_4 = torch.load('temp.pt')
    break
    

Restore this hidden node, because it can not weight-tuning to an acceptable SLFN
